# 슈퍼마켓의 판매데이터 전처리 

## 1. 데이터로드 

In [1]:
import pandas as pd 

# 파일 데이터 로드 
data1 = pd.read_csv('../../datasets/판매내역.csv', encoding="utf-8")
data2 = pd.read_csv('../../datasets/공급자.csv', encoding="utf-8")
data3 = pd.read_csv('../../datasets/상품정보.csv', encoding="cp949", sep="|")
data4 = pd.read_csv('../../datasets/소분류.csv', encoding="cp949")
data5 = pd.read_csv('../../datasets/중분류.csv', encoding="cp949")
data6 = pd.read_csv('../../datasets/대분류.csv', encoding="cp949")

## 2. 작업전 데이터 정비(결측치 보정, 데이터 타입 변경)

In [2]:
# 필드 조정 null 제거 및 필드명 변경
data3.isnull().sum()
data3 = data3.dropna(subset=["분류1", "분류2"]) #분류1" 또는 "분류2" 열 중 하나라도 값이 누락된 행을 삭제
data3.rename(columns={"분류1":"소분류코드"}, inplace=True)
data3.rename(columns={"분류2":"공급자코드"}, inplace=True)
# 필드 타입 변경
data3.astype({"소분류코드":"int64"})
data3.astype({"공급자코드":"int64"})

,상품코드,상품명,수량,소분류코드,공급자코드,"등록일시,,"
0,2100000626052,송염고은단치약130*3,NaN,20301.0,1042,"2021-01-26 18:01:54,,"
1,2100000626069,2080어린이딸기향80*2,3입,20101.0,6007,"2021-01-26 18:01:54,,"
2,2100000626076,LG토디앙황토특대/40p,40p,20402.0,1051,"2021-01-26 18:01:54,,"
3,2100000626083,참기름자반볶음번들,50g+50g,10301.0,9999,"2021-01-26 18:01:54,,"
4,2100000613007,할라페뇨슬라이스,540g,10903.0,9999,"2021-01-26 18:01:54,,"
...,...,...,...,...,...,...
34769,2100000111848,버/수퍼슬립레드보루,NaN,60102.0,7777,"2021-01-26 18:01:54,,"
34770,2100000111855,델)프/오렌지*2병,NaN,10201.0,5097,"2021-01-26 18:01:54,,"
34771,2100000106158,캡휴지걸이,NaN,20301.0,5014,"2021-01-26 18:01:54,,"
34772,2100000106165,커피향초중,NaN,20301.0,5014,"2021-01-26 18:01:54,,"


## 3. 데이터를 조인(연결) 및 정리 

In [3]:
# 데이터의 innerjoin 대분류 + 중분류
mdata = pd.merge(left = data5, right = data6, how="inner", on="대분류코드")

In [4]:
# 데이터의 innerjoin (대분류 + 중분류) +소분류
classData = pd.merge(left=mdata, right=data4, how="inner", on="중분류코드")

In [5]:
# 데이터의 innerjoin 분류코드 + 상품정보
product = pd.merge(left=classData, right=data3, how="inner", on="소분류코드")

In [6]:
# 시각화에 사용되지 않는 필드 삭제 
product = product.drop(['등록일시','대분류코드', '중분류코드', '소분류코드', '등록일시,,', '수량'], axis="columns")

In [7]:
# 데이터의 innerjoin 상품정보 + 공급자
productDetail = pd.merge(left=product, right=data2, how="inner", on="공급자코드")

In [8]:
# 시각화에 사용되지 않는 필드 삭제
productDetail = productDetail.drop(["공급자코드"], axis="columns")

In [9]:
#데이터 타입 변경
data1 = data1.astype({"판매일시":"datetime64[ns]"})
productDetail = productDetail.astype({"상품코드":"int64"})

In [10]:
# 데이터의 innerjoin 상품데이터 + 판매데이터
salesData = pd.merge(left=productDetail, right=data1, how="inner", on="상품코드")

In [11]:
# 시각화에 사용되지 않는 필드 삭제
salesData = salesData.drop(["상품코드"], axis="columns")

## 4. 년도 생성

In [12]:
# 새로운 데이터 판매년도를 생성
salesData['판매년도'] = salesData['판매일시'].dt.year

In [13]:
salesData.dtypes

중분류명                        object
대분류명                        object
소분류명                        object
상품명                         object
공급자명                        object
판매일시                datetime64[ns]
판매횟수                         int64
판매량                          int64
판매금액                         int64
판매년도                         int32
dtype: object

## 5. 작업 결과를 csv 파일로 저장

In [15]:
salesData.to_csv('../../datasets/판매정보.csv', encoding='utf-8-sig', index_label='순서')